In [1]:
import random
import threading

from sko.PSO import PSO
from math import exp

import pandas as pd
import numpy as np
from scipy.special import gamma as GAMMA

In [2]:
def get_fractional_series(X0, zeta):
    """
    计算序列的分数阶
    :param X0: 待计算分数阶的序列
    :param zeta: zeta
    :return: 分数阶序列
    """
    len_x0 = len(X0)

    # 初始化数组
    xishu = np.zeros((len_x0, len_x0))
    Xr = np.zeros(len_x0)

    for k in range(len_x0):
        tmp = 0
        for i in range(k + 1):
            xishu[k, i] = GAMMA(zeta + k - i) / (GAMMA(k - i + 1) * GAMMA(zeta))
            tmp += xishu[k, i] * X0[i]
        Xr[k] = tmp

    return Xr


def func(XX):
    zeta, tau, gamma = XX

    # 第一列是年份，故先舍弃
    X0 = data.iloc[:train_n, 1:].to_numpy()

    X1 = np.array([get_fractional_series(X0[:, 0], zeta),
                   get_fractional_series(X0[:, 1], zeta),
                   get_fractional_series(X0[:, 2], zeta)]).T

    y1 = X1[:, 0] ** (1 - gamma)

    Zy = (y1[1:] + y1[:-1]) / 2
    M2 = (X1[1:, 1] + X1[:-1, 1]) / 2
    M3 = (X1[1:, 2] + X1[:-1, 2]) / 2

    # 构建 B
    TAU = (np.exp(tau) - 1) / tau * np.exp(tau * (np.arange(1, train_n)))
    ONE = np.ones(train_n - 1)
    B = (1 - gamma) * np.array([-Zy, M2, M3, TAU, ONE]).T

    # 构建 Y
    Y = np.diff(y1)

    # 最小二乘法
    # 最小二乘法
    try:
        a, b2, b3, h1, h2 = np.dot(np.dot(np.linalg.inv(np.dot(B.T, B)), B.T), Y)
    except:
        return 1e9 # 有时会出现几万的mape，如果异常则返回1亿，不会对训练造成冲突

    L2 = 0.001 * (a ** 2 + b2 ** 2 + b3 ** 2)
    # 求 mu
    mu1 = (1 - gamma) / (1 + 0.5 * a * (1 - gamma))
    mu2 = (1 - 0.5 * a * (1 - gamma)) / (1 + 0.5 * a * (1 - gamma))
    mu3 = (h1 * (1 - gamma) * (exp(tau) - 1)) / (1 + 0.5 * a * (1 - gamma)) / tau
    mu4 = (h2 * (1 - gamma)) / (1 + 0.5 * a * (1 - gamma))

    # 这里重新获取下原数据，以实现所有数据的预测
    # 第一列是年份，故先舍弃
    X0 = data.iloc[:, 1:].to_numpy()
    X1 = np.array([get_fractional_series(X0[:, 0], zeta),
                   get_fractional_series(X0[:, 1], zeta),
                   get_fractional_series(X0[:, 2], zeta)]).T

    # 求 hat y1(1为上标的那个haty^1)
    hat_y1 = [y1[0]]
    a = []
    for m in range(2, n + 1):
        hat_ym = 0
        hat_ym += mu2 ** (m - 1) * y1[0]
        for v in range(1, m):
            a.append(m - v)
            hat_ym += mu2 ** (v - 1) * mu1 * (b2 * X1[m - v, 1] + b3 * X1[m - v, 2])
        for w in range(0, m - 1):
            hat_ym += mu2 ** w * (mu4 + mu3 * exp(tau * (m - w - 2)))
        hat_y1.append(hat_ym)

    hat_x1 = np.array(hat_y1) ** (1 / (1 - gamma))

    hat_x0 = []
    for k in range(2, n + 1):
        hat_x0_m = 0
        for j in range(0, k):
            hat_x0_m += (-1) ** j * GAMMA(zeta + 1) / (GAMMA(j + 1) * GAMMA(zeta - j + 1)) * hat_x1[k - j - 1]
        hat_x0.append(hat_x0_m)

    original_x1 = data.iloc[1:, 1]
    APE = np.abs(hat_x0 - original_x1) / original_x1
    if APE.isnull().any():
        return 1e18 # 有时会出现几万的mape，如果异常则返回1亿，不会对训练造成冲突
    else:
        return np.sqrt(np.mean(APE**2))+L2

In [3]:
def target():
    for i in range(10):
        pop = random.randint(1,200)
        max_iter = random.randint(1,1500)

        # print(f'{cnt} 的 参数为 pop: {pop}, max_iter: {max_iter}')

        # 调用粒子群，指定上述五个变量的取值范围，设置粒子群参数，最终得到最优的 zeta, tau, gamma
        pso = PSO(func=func, n_dim=3, pop=pop, max_iter=max_iter, lb=[0, -10, 0], ub=[1, 10, 10])
        pso.record_mode = True
        pso.run()


        result.append((pop, max_iter, pso.gbest_x[0], pso.gbest_x[1], pso.gbest_x[2], pso.gbest_y[0]))

        # print(cnt, end=' ')
        print((pop, max_iter, pso.gbest_x[0], pso.gbest_x[1], pso.gbest_x[2], pso.gbest_y[0]))

In [4]:
result = []

data = pd.read_excel("使用版2012-2023-2个相关因素_2位小数.xlsx")
data = data.iloc[:, :]
n = data.shape[0]
train_n = n-2

In [ ]:
# 创建多个线程
threads = [threading.Thread(target=target) for _ in range(20)]

# 启动线程
for thread in threads:
    thread.start()

# 等待所有线程完成
for thread in threads:
    thread.join()

D:\Soft\Python\Anaconda\envs\common\lib\site-packages\ipykernel_launcher.py:81: RuntimeWarning: invalid value encountered in power
D:\Soft\Python\Anaconda\envs\common\lib\site-packages\ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in double_scalars
  app.launch_new_instance()


(3, 1303, 1.0, -0.2379229866144717, 5.5085863982352645, 0.7121371228889869)
(10, 507, 0.014186654076858086, -1.0, 0.0, 0.06278850674314712)
(32, 391, 0.010742967670779195, 0.20296442474737353, 4.3857043379477165, 0.026854699619630294)
(105, 49, 0.014186284606903249, -1.0, 0.0, 0.06278850674397043)
(17, 1055, 0.0618094650937269, 0.05349107603705268, 5.731230950470831, 0.059435971630046866)
(88, 291, 0.016090820863523227, 0.19743024207321042, 4.702405466177799, 0.026936610880745587)
(44, 693, 0.014186627552999924, -1.0, 0.0, 0.06278850674306057)
(89, 344, 0.01418664916363996, -1.0, 0.0, 0.06278850674313372)
(166, 215, 0.024664486965343964, -0.6730724036066972, 8.800710150240366, 0.03608324016206075)
(20, 1414, 0.01418666565616154, -1.0, 0.0, 0.0627885067431312)
(38, 927, 0.014186620916600116, -1.0, 0.0, 0.06278850674310335)
(172, 225, 2.1605020454671623e-09, 0.3349047034946189, 0.0, 0.03539740129721336)


D:\Soft\Python\Anaconda\envs\common\lib\site-packages\numpy\lib\function_base.py:1269: RuntimeWarning: invalid value encountered in subtract
  a = op(a[slice1], a[slice2])


(53, 253, 0.014186593504893064, -1.0, 0.0, 0.0627885067431143)
(109, 521, 7.040239156571103e-15, 0.33490446901761356, 0.0, 0.03447375749793204)
(80, 1049, 0.007693826333639437, -0.7201684971779229, 5.944809165059564, 0.033854450163990704)
(74, 1141, 5.253685423760252e-15, 0.33479577014294426, 0.0, 0.034154254229687514)
(129, 728, 0.9159059067733208, 0.22214978081649278, 0.10337163221813475, 0.039228418682101966)
(3, 672, 0.012656531898449763, -1.0, 10.0, 0.10436422513776972)
(160, 641, 0.0077048290108465, -0.7202808130239053, 5.9453525648476075, 0.033854765564134054)
(156, 822, 0.014186612764404216, -1.0, 0.0, 0.06278850674309903)
(56, 1452, 0.014186610369242748, -1.0, 0.0, 0.06278850674312556)
(173, 777, 0.010112680548265208, 0.19951865487719458, 4.624867238258501, 0.02652185328133308)
(12, 282, 0.016775586501009774, 0.34941880204517756, 0.6465331640166208, 0.04530800864460143)
(106, 1304, 0.01418666317894992, -1.0, 0.0, 0.06278850674308997)
(138, 86, 7.88638874714241e-08, 0.334900776

In [ ]:
# with ThreadPoolExecutor(max_workers=100) as executor:
#     futures = []
#     for i in range(100):
#         # 提交任务到线程池
#         future = executor.submit(target, i)
#         futures.append(future)
#
#     # 等待所有线程（任务）完成
#     for future in futures:
#         print("线程数量： ", executor.work_count())
#         future.result()

In [ ]:
# threads = []
# for i in range(1000):
#     t = threading.Thread(target=target)
#     t.start()
#     threads.append(t)
#
# # 等待所有线程完成
# for t in threads:
#     t.join()
#
# pd.DataFrame(result).to_excel('file/01.xlsx')

In [ ]:
pd.DataFrame(result).to_excel('case1.xlsx')